C:\Users\djorgens\AppData\Local\Temp\ipykernel_17496\4063275191.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_jobs['Company'] = final_jobs['Company'].replace(['Genesis Health Systems'], 'Genesis Health System')
C:\Users\djorgens\AppData\Local\Temp\ipykernel_17496\4063275191.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_jobs['Empl_type']=final_jobs['Empl_type'].fillna('Full-Time/Part-Time')
C:\Users\djorgens\AppData\Local\Temp\ipykernel_17496\4063275191.py:35: SettingWithCopyWarning

In [ ]:

import pandas as pd
from functools import reduce
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import sys
sys.path.insert(0, 'C:\Projects\Staffing')

#-----------------
# Job Descriptions
#-----------------
jf_df = pd.read_csv("C:\Projects\Staffing/data/Combined_Jobs_Final.csv")

cols = ['Job.ID', 'Title', 'Position', 'Company', 'City', 'State.Name', 'Job.Description', 'Employment.Type', 'Education.Required']
final_jobs = jf_df[cols]
final_jobs.columns = ['Job.ID', 'Title', 'Position', 'Company', 'City', 'State Name', 'Job_Description', 'Empl_type', 'Edu_req']

#replacing nan with thier headquarters location
final_jobs['Company'] = final_jobs['Company'].replace(['Genesis Health Systems'], 'Genesis Health System')
final_jobs.loc[final_jobs.Company == 'CHI Payment Systems', 'City'] = 'Illinois'
final_jobs.loc[final_jobs.Company == 'Academic Year In America', 'City'] = 'Stamford'
final_jobs.loc[final_jobs.Company == 'CBS Healthcare Services and Staffing ', 'City'] = 'Urbandale'
final_jobs.loc[final_jobs.Company == 'Driveline Retail', 'City'] = 'Coppell'
final_jobs.loc[final_jobs.Company == 'Educational Testing Services', 'City'] = 'New Jersey'
final_jobs.loc[final_jobs.Company == 'Genesis Health System', 'City'] = 'Davennport'
final_jobs.loc[final_jobs.Company == 'Home Instead Senior Care', 'City'] = 'Nebraska'
final_jobs.loc[final_jobs.Company == 'St. Francis Hospital', 'City'] = 'New York'
final_jobs.loc[final_jobs.Company == 'Volvo Group', 'City'] = 'Washington'
final_jobs.loc[final_jobs.Company == 'CBS Healthcare Services and Staffing', 'City'] = 'Urbandale'

# add data into last NAN's
final_jobs['Empl_type']=final_jobs['Empl_type'].fillna('Full-Time/Part-Time')

# Creating the corpus of the jobs
final_jobs["pos_com_city_empType_jobDesc_title"] = final_jobs["Position"].map(str) + " " + final_jobs["Company"] +" "+ final_jobs["City"]+ " "+final_jobs['Empl_type']+" "+final_jobs['Job_Description']+" " + final_jobs['Title']

#removing unnecessary characters and convert to lowercase
final_jobs['pos_com_city_empType_jobDesc_title'] = final_jobs['pos_com_city_empType_jobDesc_title'].str.replace('[^a-zA-Z \n\.]'," ").str.lower() 

#print(final_jobs['pos_com_city_empType_jobDesc_title'].head(1000))
final_jobs = final_jobs[['Job.ID', 'pos_com_city_empType_jobDesc_title']]

# Creating the corpus of the applications




In [92]:
#-----------------
# Position of interest
#-----------------
poi = pd.read_csv("C:\Projects\Staffing/data/Positions_Of_Interest.csv", sep=',')
poi = poi.sort_values(by='Applicant.ID')
# drop unneeded columns
poi = poi.drop('Updated.At', 1)
poi = poi.drop('Created.At', 1)

#cleaning the text
poi['Position.Of.Interest']=poi['Position.Of.Interest'].str.replace('[^a-zA-z \n\.]',"")
poi['Position.Of.Interest']=poi['Position.Of.Interest'].str.lower()
poi = poi.fillna(" ")
#print(poi.head(20))

poi = poi.groupby('Applicant.ID', sort=True)['Position.Of.Interest'].apply(' '.join).reset_index()
#print(poi.head(20))

# Experiences
exper_applicant = pd.read_csv("C:\Projects\Staffing/data/Experience.csv")
#print(exper_applicant.head())
#taking only Position
exper_applicant = exper_applicant[['Applicant.ID','Position.Name', 'Job.Description']]

exper_applicant.dropna(axis=0,inplace=True)
print(exper_applicant.isnull().sum())
print(exper_applicant.shape)
#cleaning the text
exper_applicant['Position.Name'] = exper_applicant['Position.Name'].str.replace('[^a-zA-Z \n\.]',"").str.lower()
exper_applicant['Job.Description'] = exper_applicant['Job.Description'].str.replace('[^a-zA-Z \n\.]',"").str.lower()

#print(exper_applicant.head())

exper_applicant = exper_applicant.sort_values(by='Applicant.ID')
exper_applicant = exper_applicant.fillna(" ")

#print(exper_applicant.head())

#adding same rows to a single row
exper_applicant_position = exper_applicant.groupby('Applicant.ID', sort=False)['Position.Name'].apply(' '.join).reset_index()
exper_applicant_desc = exper_applicant.groupby('Applicant.ID', sort=False)['Job.Description'].apply(' '.join).reset_index()
#print(exper_applicant_position.head(20))
#print(exper_applicant_desc.head(20))
data_frames = [poi, exper_applicant_position, exper_applicant_desc]
final_experience = reduce(lambda  left, right: pd.merge(left,right,on=['Applicant.ID'],
                                            how='outer'), data_frames).fillna(" ")

final_experience["poi_pos_desc"] = final_experience["Position.Of.Interest"].map(str) + " " + final_experience["Position.Name"] +" "+ final_experience["Job.Description"]
final_experience = final_experience[['Applicant.ID', 'poi_pos_desc']]
print(final_experience.shape)
print(final_experience.head())

C:\Users\djorgens\AppData\Local\Temp\ipykernel_17496\2164845249.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  poi = poi.drop('Updated.At', 1)
C:\Users\djorgens\AppData\Local\Temp\ipykernel_17496\2164845249.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  poi = poi.drop('Created.At', 1)
C:\Users\djorgens\AppData\Local\Temp\ipykernel_17496\2164845249.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  poi['Position.Of.Interest']=poi['Position.Of.Interest'].str.replace('[^a-zA-z \n\.]',"")
C:\Users\djorgens\AppData\Local\Temp\ipykernel_17496\2164845249.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  exper_applicant['Position.Name'] = exper_applicant['Position.Name'].str.replace('[^a-zA-Z \n\.]',"").str.

Applicant.ID       0
Position.Name      0
Job.Description    0
dtype: int64
(5668, 3)


C:\Users\djorgens\AppData\Local\Temp\ipykernel_17496\2164845249.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  exper_applicant['Job.Description'] = exper_applicant['Job.Description'].str.replace('[^a-zA-Z \n\.]',"").str.lower()


(3259, 2)
   Applicant.ID                                       poi_pos_desc
0            96  server cashietwaiter cashier receptionist plac...
1           153  barista host server sales rep customer service...
2           256  host production area sales rep customer servic...
3           438       customer service rep barista host server    
4           568  receptionist customer service rep book keeper ...


In [94]:
stop = stopwords.words('english')
final_experience["poi_pos_desc"] = final_experience["poi_pos_desc"].astype(str)
final_jobs["pos_com_city_empType_jobDesc_title"] = final_jobs["pos_com_city_empType_jobDesc_title"].astype(str)

# for experiences 
only_text_experience = final_experience["poi_pos_desc"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
only_text_experience = only_text_experience.apply(lambda x : filter(None,x.split(" ")))
#stemmer =  PorterStemmer()
#only_text_experience = only_text_experience.apply(lambda x : [stemmer.stem(y) for y in x])
only_text_experience = only_text_experience.apply(lambda x : " ".join(x))
final_experience["text"] = only_text_experience

# for jobs 
only_text_jobs = final_jobs["pos_com_city_empType_jobDesc_title"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
only_text_jobs = only_text_jobs.apply(lambda x : filter(None,x.split(" ")))
#stemmer =  PorterStemmer()
#only_text_jobs = only_text_jobs.apply(lambda x : [stemmer.stem(y) for y in x])
only_text_jobs = only_text_jobs.apply(lambda x : " ".join(x))


final_jobs["text"] = only_text_jobs

C:\Users\djorgens\AppData\Local\Temp\ipykernel_17496\772136335.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_jobs["pos_com_city_empType_jobDesc_title"] = final_jobs["pos_com_city_empType_jobDesc_title"].astype(str)
C:\Users\djorgens\AppData\Local\Temp\ipykernel_17496\772136335.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_jobs["text"] = only_text_jobs


In [95]:
import nltk
nltk.download('punkt')
print(final_jobs.head())

   Job.ID                 pos_com_city_empType_jobDesc_title  \
0     111  server tacolicious palo alto part time tacolic...   
1     113  kitchen staff chef claude lane san francisco p...   
2     117  bartender machka restaurants corp. san francis...   
3     121  server teriyaki house brisbane part time    se...   
4     127  kitchen staff chef rosa mexicano   sunset los ...   

                                                text  
0  server tacolicious palo alto part time tacolic...  
1  kitchen staff chef claude lane san francisco p...  
2  bartender machka restaurants corp. san francis...  
3  server teriyaki house brisbane part time serve...  
4  kitchen staff chef rosa mexicano sunset los an...  


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\djorgens\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
from nltk.tokenize import word_tokenize
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(final_jobs["text"].tolist())]

In [ ]:
from gensim.models.doc2vec import Doc2Vec,TaggedDocument

max_epochs = 100
vec_size = 20
alpha = 0.025



#print(tagged_data)

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=100,
                dm =1)
  


In [ ]:
model.build_vocab(tagged_data) # tagged_data

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data, # tagged_data
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

In [96]:
# Test inferencing:

model= Doc2Vec.load("d2v.model")
print("Model loaded!")

#inferred_vectors_test = [model.infer_vector(doc) for doc in word_tokenize(final_experience["text"])]
inferred_vectors_test = []
for doc in final_experience["text"].tolist():
    inferred_vectors_test.append(model.infer_vector(word_tokenize(doc)))

# now we have a vector for each test-document (All the Applicant experiences!)
#print(inferred_vectors_test)
import csv


with open("doc2vec_jobmatchings.csv", "w", newline='') as f:
    writer = csv.writer(f)

    #print(tagged_data[10])
    #print(final_jobs["pos_com_city_empType_jobDesc_title"][10])
    #print(final_jobs["Job.ID"][10])
    csv_input = ['match_pct','Job.ID','pos_com_city_empType_jobDesc_title' , 'poi_pos_desc']
    writer.writerows([csv_input])

    applicant = 0
    for test in inferred_vectors_test:
        best_jobs = model.docvecs.similar_by_vector(test)
        best_match = best_jobs[0]
        #print(best_match)
        #print(final_jobs["pos_com_city_empType_jobDesc_title"][int(best_match[0])])
        csv_input = [str(best_match[1]), str(best_match[0]), f"{final_jobs['text'][int(best_match[0])]}", f"{final_experience['text'][applicant]}"]
        
        writer.writerows([csv_input])
        applicant +=1


Model loaded!


C:\Users\djorgens\AppData\Local\Temp\ipykernel_17496\3847254524.py:27: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  best_jobs = model.docvecs.similar_by_vector(test)


In [25]:
#initializing tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

tfidf_jobid = tfidf_vectorizer.fit_transform((final_jobs['text'])) #fitting and transforming the vector
print(tfidf_jobid)

from sklearn.metrics.pairwise import cosine_similarity
user_tfidf = tfidf_vectorizer.transform(final_experience['text'])
output = map(lambda x: cosine_similarity(user_tfidf, x), tfidf_jobid)




  (0, 41881)	0.06109257352435258
  (0, 29266)	0.05223200207781344
  (0, 45241)	0.051986473404771964
  (0, 41468)	0.11452622039386127
  (0, 12164)	0.10452184584098292
  (0, 7903)	0.06968300001929177
  (0, 28984)	0.04061333598167795
  (0, 6632)	0.10569960514866913
  (0, 492)	0.08409004000047433
  (0, 26302)	0.07095302471785131
  (0, 34750)	0.050396037022325486
  (0, 10449)	0.042067526139897074
  (0, 13247)	0.1420593777145455
  (0, 16928)	0.08038712091445505
  (0, 39813)	0.06505131370808943
  (0, 37724)	0.0990607682290021
  (0, 48678)	0.03315715511256172
  (0, 43416)	0.22328322655140706
  (0, 25615)	0.17378690885952303
  (0, 19748)	0.06515641021738212
  (0, 36251)	0.08279681880931611
  (0, 30669)	0.06582819439127076
  (0, 42317)	0.062407659487140166
  (0, 16582)	0.07476217646452103
  (0, 44643)	0.02874471822006935
  :	:
  (84088, 43714)	0.04018960689076876
  (84088, 6181)	0.05617621468473928
  (84088, 43449)	0.06084095104263497
  (84088, 12501)	0.11863771843955294
  (84088, 16517)	0.07109

In [38]:
output2 = list(output)
len(output2)

from gensim.models.doc2vec import Doc2Vec,TaggedDocument

model= Doc2Vec.load("d2v.model")
print("Model loaded!")

Model loaded!


In [32]:
# random user
u = 300

#getting the job id's of the recommendations
top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:50]
recommendation = pd.DataFrame(columns = ['ApplicantID', 'JobID', 'pos_com_city_empType_jobDesc_title'])
count = 0
for i in top:
    recommendation.set_value(count, 'ApplicantID', u)
    recommendation.set_value(count,'JobID' ,final_jobs['Job.ID'][i])
    recommendation.set_value(count,'JobID' ,final_jobs['pos_com_city_empType_jobDesc_title'][i])
    count += 1

In [33]:
print(recommendation.head())

Empty DataFrame
Columns: [ApplicantID, JobID, pos_com_city_empType_jobDesc_title]
Index: []
